In [3]:
import pandas as pd

In [4]:
df15 = pd.read_csv('dataset/2015.csv', encoding='cp1252', sep=';')
df16 = pd.read_csv('dataset/2016.csv', encoding='cp1252', sep=';')
df17 = pd.read_csv('dataset/2017.csv', encoding='cp1252', sep=';')
df18 = pd.read_csv('dataset/2018.csv', encoding='cp1252', sep=';')
df19 = pd.read_csv('dataset/2019.csv', encoding='cp1252', sep=';')
df20 = pd.read_csv('dataset/2020.csv', encoding='cp1252', sep=';')
df21 = pd.read_csv('dataset/2021.csv', encoding='cp1252', sep=';')
df22 = pd.read_csv('dataset/2022.csv', encoding='cp1252', sep=';')
df23 = pd.read_csv('dataset/2023.csv', encoding='cp1252', sep=';')
df24 = pd.read_csv('dataset/2024.csv', encoding='cp1252', sep=';')

In [5]:
df = pd.concat([df15, df16, df17, df18, df19, df20, df21, df22, df23, df24])

In [6]:
desastres = list(df['COBRADE'].unique())

In [66]:
id_full_disasters = {}
for d in desastres:
    cod = d[:5]
    id_full_disasters[cod] = d[8:]

In [7]:
keep_cols = [
    'UF', 'Municï¿½pio', 'Registro', 'COBRADE',
    'Status', 'Populaï¿½ï¿½o', 'DH_Mortos', 'DH_Feridos', 'DH_Enfermos',
    'DH_Desabrigados', 'DH_Desalojados'
]
df = df[keep_cols]
df['DH_Feridos_Enfermos'] = df['DH_Feridos'] + df['DH_Enfermos']
df['DH_Desabrigados_Desalojados'] = df['DH_Desabrigados'] + df['DH_Desalojados']
df = df.drop(['DH_Feridos', 'DH_Enfermos', 'DH_Desabrigados', 'DH_Desalojados'], axis=1)

In [8]:
df = df.rename(columns={'Municï¿½pio': 'Municipio', 'COBRADE': 'Desastre', 'Populaï¿½ï¿½o': 'Populacao'})
df['Desastre'] = df['Desastre'].str[:5]
df['Data'] = pd.to_datetime(df['Registro'], format='%d/%m/%Y')
df = df.drop(['Registro'], axis=1)
df = df.reset_index(drop=True)

In [9]:
df

,UF,Municipio,Desastre,Status,Populacao,DH_Mortos,DH_Feridos_Enfermos,DH_Desabrigados_Desalojados,Data
0,RJ,Duque de Caxias,13215,Registro,855046,0.0,0.0,0.0,2016-01-05
1,RS,Manoel Viana,12100,Reconhecido,7074,0.0,0.0,280.0,2015-12-30
2,RS,Paraï¿½so do Sul,12100,Reconhecido,7336,0.0,0.0,0.0,2015-12-30
3,SC,ï¿½guas Mornas,12200,Registro,5546,0.0,0.0,0.0,2016-01-05
4,RS,Sï¿½o Vicente do Sul,13214,Reconhecido,8440,0.0,0.0,0.0,2015-12-30
...,...,...,...,...,...,...,...,...,...
64057,PR,Morretes,15110,Reconhecido,15718,0.0,254.0,0.0,2024-04-05
64058,CE,Missï¿½o Velha,13214,Registro,34258,1.0,0.0,0.0,2024-01-10
64059,BA,Serra do Ramalho,14110,Registro,31646,0.0,0.0,0.0,2024-01-18
64060,RN,Porto do Mangue,11410,Registro,5217,0.0,0.0,0.0,2024-01-01


## 2. Computation of Frequent Eventsets:

- Remove "itemsets" with "Desastre" type count less than min_sup

In [96]:
min_sup = 0.5/100
count = dict(df['Desastre'].value_counts())
tot = sum(count.values())
disasters_to_drop = set()

for d, c in count.items():
    if c/tot < min_sup:
        disasters_to_drop.add(d)

# doencas virais
disasters_to_drop.add('15110')

df_min_sup = df[~df['Desastre'].isin(disasters_to_drop)]

## 3. Calculating frequent sequences

1. With pycspade
2. Transform into:

```
Your data needs to be in a particular format similar to the following:

1 1 3 8 37 42
1 2 4 4 11 37 42
2 1 2 10 73
2 2 1 72
2 3 3 4 24 77
...
The first number is the sequence index, the second is the event index, the third is the number of elements, followed by the element, space separated

data = [
    [1, 10, [3, 4]],
    [1, 15, [1, 2, 3]],
    [1, 20, [1, 2, 6]],
    [1, 25, [1, 3, 4, 6]],
    [2, 15, [1, 2, 6]],
    [2, 20, [5]],
    [3, 10, [1, 2, 6]],
    [4, 10, [4, 7, 8]],
    [4, 20, [2, 6]],
    [4, 25, [1, 7, 8]]
]

To construct this:
1. Identify all unique items: Extract all unique values from your Desastre column.
2. Populate id-lists: For each unique Desastre item, create a list of (sid, eid) pairs where that item appeared.
Here’s an illustrative partial example of your data in vertical format:

```

In [97]:
d_ids = {}
for id, d in enumerate(df_min_sup['Desastre'].unique()):
    d_ids[d] = id + 1

dates_ids = {}
for id, d in enumerate(df_min_sup.sort_values('Data')['Data'].unique()):
    dates_ids[str(d.astype('datetime64[D]'))] = id + 1

df_min_sup['UF_Municipio'] = df_min_sup['UF'].astype(str) + '-' + df_min_sup['Municipio']
uf_mun_ids = {}
for id, uf_mun in enumerate(df_min_sup['UF_Municipio'].unique()):
    uf_mun_ids[uf_mun] = id + 1

/tmp/ipykernel_114648/653276908.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_min_sup['UF_Municipio'] = df_min_sup['UF'].astype(str) + '-' + df_min_sup['Municipio']


In [98]:
vertical_dataset = {}
for uf_mun in df_min_sup['UF_Municipio'].unique():
    events = pd.DataFrame(df_min_sup[df_min_sup['UF_Municipio'] == uf_mun])
    events['Data'] = events['Data'].astype('datetime64[D]')
    items = events[['Data', 'Desastre']]
    vertical_dataset[uf_mun] = items

In [99]:
max_id = 0

spade_input = []
for uf_city, events in vertical_dataset.items():
    entries = []
    for date in events.sort_values('Data')['Data'].unique():
        disasters = list(events[events['Data'] == date]['Desastre'].values)
        np_date = str(pd.Timestamp(date).to_numpy().astype('datetime64[D]'))
        entries.append([uf_mun_ids[uf_city], dates_ids[np_date], disasters])
    spade_input += entries

In [15]:
from pycspade.helpers import spade, print_result

In [110]:
result = spade(data=spade_input, support=0.15)
print_result(result)

fmt_rules = []

for rule in result['mined_objects']:
    fmt_rules.append(list(map(lambda x: id_full_disasters[str(x.__dict__['elements'][0])], rule.__dict__['items'])))

fmt_rules

   Occurs     Accum   Support    Confid      Lift                                                                         Sequence
      861       861 0.1808444       N/A       N/A                                                                          (12100) 
     1109      1109 0.2329343       N/A       N/A                                                                          (12200) 
      784       784 0.1646713 0.7069432 1.2042063                                                                 (12200)->(13214) 
      794       794 0.1667717       N/A       N/A                                                                          (12300) 
     2795     11164 0.5870615       N/A       N/A                                                                          (13214) 
     1690      3518 0.3549674 0.6046512 1.0299621                                                                 (13214)->(13214) 
     1012      1012 0.2125604 0.5988166 1.0200235                            

[['Inundaï¿½ï¿½es'],
 ['Enxurradas'],
 ['Enxurradas', 'Tempestade Local/Convectiva - Chuvas Intensas'],
 ['Alagamentos'],
 ['Tempestade Local/Convectiva - Chuvas Intensas'],
 ['Tempestade Local/Convectiva - Chuvas Intensas',
  'Tempestade Local/Convectiva - Chuvas Intensas'],
 ['Tempestade Local/Convectiva - Chuvas Intensas',
  'Tempestade Local/Convectiva - Chuvas Intensas',
  'Tempestade Local/Convectiva - Chuvas Intensas'],
 ['Tempestade Local/Convectiva - Chuvas Intensas', 'Estiagem'],
 ['Tempestade Local/Convectiva - Vendaval'],
 ['Tempestade Local/Convectiva - Vendaval',
  'Tempestade Local/Convectiva - Chuvas Intensas'],
 ['Estiagem'],
 ['Estiagem', 'Tempestade Local/Convectiva - Chuvas Intensas'],
 ['Estiagem',
  'Tempestade Local/Convectiva - Chuvas Intensas',
  'Tempestade Local/Convectiva - Chuvas Intensas'],
 ['Estiagem', 'Estiagem'],
 ['Estiagem', 'Estiagem', 'Tempestade Local/Convectiva - Chuvas Intensas'],
 ['Estiagem', 'Estiagem', 'Estiagem'],
 ['Estiagem',
  'Estiagem'

## 3. Subgroup Discovery